# MPLS generator performance
This notebook is intended to evaluate the performance of the data plane generator in MPLS-Kit.
It generates all topologies at random, with one dataplane per topology. The elapsed time in the creation of the topology is ignored, just the time for data plane construction is accounted for.

*WARNING: This notebook might take several minutes to complete its run.*

In [ ]:
#Requirements

import networkx as nx
import matplotlib.pyplot as plt
import random
import time
import json
import math 
import numpy as np
from pprint import pprint
from itertools import chain, count

from networkx.algorithms.shortest_paths.weighted import _weight_function, _dijkstra_multisource
from resource import getrusage, RUSAGE_SELF

from mpls_fwd_gen import *

# import pandas as pd
import os

## Demo parameters

In [ ]:
n_vect = [20, 40, 60, 80, 100]
n_vect = np.array(n_vect)

num_lsps_vect = n_vect * 3
num_services_vect = n_vect*2

repetitions_per_size = 100

assert len(n_vect) == len(num_lsps_vect) == len(num_services_vect)

# Topology configuration

num_topos = len(n_vect)

# Random
mode = "random.log_degree"   # custom, random.log_degree, random.large_degree
n = 10                     # number of nodes, for random only
weight_mode = "random"       # random or equal or distance
gen_method = 1               # method for generating the random topology
#     visualize = True if n<60 else False         # Use only with small networks


visualize = False         # Use only with small networks
display_tables = False       # Use only with small networks


# Generator configuration
enable_PHP = False
numeric_labels = False

enable_LDP = True
enable_RSVP = True
# num_lsps = 20           # Maximum number of LSPS to compute for RSVP only
# protection = None
protection = "facility-node" # "facility-node"
# num_lsps = [("R1","R4")]        

# add a param for how many tunnels between same head / endtail with the exact same path
tunnels_per_pair = 1

enable_services = True
# num_services = 15
PE_s_per_service = 5
CEs_per_PE = 1


enable_RMPLS = False

#random_seed
random_seed = 1

## Select topology and generate MPLS forwarding rules

In [ ]:
e_vect = []
t_vect = np.empty((repetitions_per_size, num_topos))

for i in range(num_topos):
    n = n_vect[i]
    print(f"  CREATE topos with {n} nodes")
    num_lsps = int(num_lsps_vect[i])
    num_services = int(num_services_vect[i])
    

    for rep in range(repetitions_per_size):
        random_seed += 1
        print(f"      repetition {rep} (seed {random_seed})")


    
        G = generate_topology(mode, 
                              n, 
                              weight_mode = "random", 
                              gen_method = 1, 
                              visualize = False, 
                              display_tables = False, 
                              random_seed = random_seed 
                             )
        n = G.number_of_nodes()
        e = G.number_of_edges()
        e_vect.append(e)

        twall0 = time.time() # wall time
        t0 = time.process_time()
        ram0 = getrusage(RUSAGE_SELF).ru_maxrss
        
        network = generate_fwd_rules(G, 
                                     enable_PHP = enable_PHP, 
                                     numeric_labels = numeric_labels, 
                                     enable_LDP = enable_LDP, 
                                     enable_RSVP = enable_RSVP,
                                       num_lsps = num_lsps, 
                                       tunnels_per_pair = tunnels_per_pair,
                                     enable_services = enable_services, 
                                       num_services = num_services, 
                                       PE_s_per_service = PE_s_per_service, 
                                       CEs_per_PE = CEs_per_PE,
                                      random_seed = random_seed,
                                     protection = protection,
                                     enable_RMPLS = enable_RMPLS
                              )
        
        t1 = time.process_time()
        twall1 = time.time() # wall timeram1 = getrusage(RUSAGE_SELF).ru_maxrss
        ram1 = getrusage(RUSAGE_SELF).ru_maxrss

        t_vect[rep, i] = t1-t0

# n_vect = np.array(n_vect)
e_vect = np.array(e_vect)
t_vect = np.array(t_vect)

# print("%"*100)
# print("CPU time computing {} topologies with {} nodes on average and {} edges on average: {}".format(num_topos, n_vect.mean(), e_vect.mean(), t1-t0 ))
# print("Elapsed time computing {} topologies with {} nodes on average and {} edges on average: {}".format(num_topos, n_vect.mean(), e_vect.mean(), twall1-twall0 ))
# print("%"*100)

In [ ]:

# print("CPU time computing {} topologies with {} nodes on average and {} edges on average: {}".format(len(n_vect), n_vect.mean(), e_vect.mean(), t1-t0 ))
# print("Elapsed time computing {} topologies with {} nodes on average and {} edges on average: {}".format(len(n_vect), n_vect.mean(), e_vect.mean(), twall1-twall0 ))

print(f"""   
Protocol generation parameters:

     enable_PHP = {enable_PHP}, 
     enable_LDP = {enable_LDP}, 
     enable_RSVP = {enable_RSVP},
       num_lsps = {num_lsps}, 
       tunnels_per_pair = {tunnels_per_pair},
     enable_services = {enable_services}, 
       num_services = {num_services}, 
       PE_s_per_service = {PE_s_per_service}, 
       CEs_per_PE = {CEs_per_PE},
     protection = {protection}

""")

pprint(n_vect)
pprint(t_vect)

### Define auxiliary plot functions.

In [ ]:
fig_width_mm = 125
aspect_ratio = 1.6
fig_height_mm  = fig_width_mm / aspect_ratio
fig_size = (fig_width_mm/25.4, fig_height_mm/25.4)   #in inches
dpi = 200
fig_filename = "/tmp/computation_times.pdf"

In [ ]:
y = t_vect
title = "Computation times"

fig, ax = plt.subplots(figsize=fig_size, dpi = dpi)
plt.rc('font', size=10)


ax.set(xlabel='no. nodes in random topology', 
       ylabel='time (s)',
       title=title,
       rasterized=False,
       autoscale_on=True)

columns = []
for c in range(y.shape[1]):
    columns.append(y[:,c])




ax.boxplot(columns)

li = list(range(1, num_topos+1))
plt.xticks(li, list(n_vect), rotation=0)

### uncomment the next two lines to save the figure.
# if fig_filename:
#     plt.savefig(fig_filename, bbox_inches="tight")   # ,pad_inches=0.2
    
plt.show()    